In [1]:
import cv2
import os
import pathlib
import glob
import numpy as np
import tensorflow as tf
import random

classes = ['1', '2', '3']
path_image = 'imgs/'

class_maps = {
    0: "Pedra",
    1: "Papel",
    2: "Tesoura",
}

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

In [4]:
model = tf.keras.models.load_model('model_rps/model_rps/')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
re_lu (ReLU)                 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0     

In [8]:
def hand(hand_user, hand_robot):
    if hand_user == hand_robot:
        return "Empate"
    
    if hand_user == "Papel":
        if hand_robot == "Pedra":
            return "Usuario"
        if hand_robot == "Tesoura":
            return "Computador"
        
    if hand_user == "Pedra":
        if hand_robot == "Tesoura":
            return "Usuario"
        if hand_robot == "Papel":
            return "Computador"
        
    if hand_user == "Tesoura":
        if hand_robot == "Papel":
            return "Usuario"
        if hand_robot == "Pedra":
            return "Computador"

In [11]:
cap = cv2.VideoCapture(1)
cap = cv2.VideoCapture(0)

pred_none = None
while True:
    ret,frame = cap.read()
    frame = cv2.resize(frame, dsize=(600, 600), interpolation=cv2.INTER_CUBIC)
    
    cv2.rectangle(frame,(145,320),(400,550),(160, 175, 179),2)
    
    roi = frame[320:550, 145:400]
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    img = tf.image.rgb_to_grayscale(img)
    img = tf.image.convert_image_dtype(img, np.float32)
    img = tf.image.resize(img, (128,128))
        
    pred = model.predict(np.array([img]))
    pred_code = np.argmax(pred[0])
    pred_name = class_maps[pred_code]
    
    if pred_none != pred_name:
        robot = random.choice(['Pedra', 'Papel', 'Tesoura'])
        win = hand(pred_name, robot)

    pred_none = pred_name
    
    cv2.putText(frame, "Usuario: " + pred_name, (20, 580), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "Computador: " + robot, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "Vencedor: " + win, (20, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (230, 228, 3), 1, cv2.LINE_AA)
    
    if robot != "none":
        icon = cv2.imread(
            "imgs/{}.png".format(robot))
        icon = cv2.resize(icon, (220, 220))
        frame[50:270, 150:370] = icon
    
    cv2.imshow('Play', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()